<div style='text-align: center;font-size: 50px;color: #24aaf2;'>
  Class Scheduling
</div>

<div style='text-align: center;font-size: 35px;color: #24aaf2;'>
  Introdução
</div>

Este projeto, tem como propósito central o desenvolvimento de um agente inteligente para a resolução de um problema de satisfação de restrições (CSP).

O tema abordado é o da elaboração de um calendário escolar para várias turmas, que obedeça a um conjunto de restrições que serão detalhadas nas secções posteriores do projeto.


### Constituição do grupo:
- Pedro Ferreira (17029)
- Enmanuel Martins (16430)


<div style='text-align: center;font-size: 35px;color: #24aaf2;'>
  Formulação do Objetivo
</div>

### Objetivo:
Desenvolvimento de um sistema de agendamento de aulas para várias turmas, que atenda a um conjunto de restrições, como por exemplo:
- Todas as aulas duram 2 horas e são leccionadas em dias semanais.
- Todas as turmas têm 4 a 10 aulas por semana.
- Uma turma não tem mais de 3 aulas por dia.
- Uma turma não tem mais de 4 dias de aulas por semana.
- Apenas 1 ou 2 aulas por Manhã/Tarde.

### Possíveis Limitações:
- Restrições adicionais que possam surgir à medida que o problema é implementado, exigindo uma adaptação do sistema para acomodar novas restrições.
- O aumento do número de turmas (disciplinas) pode tornar necessário o aumento do número de professores e salas, impactando assim a  eficiência do sistema.

### Ações a serem Tomadas:
- Implementar um algoritmo de agendamento de aulas que leve em consideração as restrições formuladas.
- Testar o sistema para cenários distintos, variando o número de disciplinas, turmas, professores e salas.
- Avaliar a capacidade do sistema de atender às restrições sem comprometer a sua eficiência.
- Investigar estratégias de otimização do sistema para reduzir o impacto das limitações à medida que a complexidade aumenta.


<div style='text-align: center;font-size: 35px;color: #24aaf2;'>
  Planeamento do Agente
</div>

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Atributos PEAS
</div>


### Medidas de performance

No contexto deste problema, a performance será uma combinação de obter estados solução que obedeçam às restrições impostas com uma complexidade temporal minimamente aceitável (o agente deverá apresentar uma solução em um espaço temporal não muito longo).

### Ambiente

Como o agente é um software, o seu ambiente serão o conjunto de dados que tem acesso (o seu ambiente é puramente computacional).

### Atuadores

O agente (software) pode atuar no ambiente com escrita de ficheiros, exibição de dados na tela,...

### Sensores

O agente (software) pode receber informações com leitura de ficheiros ou outros tipos de inputs.


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Propriedades do Ambiente de Tarefas
</div>

De forma sucinta, o nosso agente terá como propriedades:
- Totalmente observável.
- Agente único.
- Determinístico (Próximo estado é determinado pelo atual e pela ação executada).
- Sequencial (A decisão atual poderá afetar decisões futuras).
- Estático (Ambiente não se altera enquanto o agente delibera a próxima ação).
- Discreto.
- Conhecido.

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Formulação do problema
</div>

Um problema de satisfação de restrições consiste em 3 componentes:
- Conjunto de variáveis.
- Conjunto de domínios para cada uma das variáveis.
- Conjunto de restrições para uma combinação válida de valores atribuídos.

Neste projeto, optámos por definir 2 turmas denominadas Lesi e Leec. As nossas variáveis serão os elementos das listas:
```python
    Lesi = ['D1_T', 'D1_TP', 'D2_T', 'D2_TP', 'D3', 'D4_T', 'D4_TP', 'D5_T', 'D5_TP']
    Leec = ['D6_T', 'D6_TP', 'D7', 'D8_T', 'D8_TP', 'D9_T', 'D9_TP', 'D10_T', 'D10_TP']
```
Utilizámos os sufixos _T e _TP para que cada disciplina tenha mais de uma aula por semana.

O domínio de cada uma das variáveis será:
```python
    list(set(range(1, 21)))
```
Ou seja, cada variável pode tomar o valor de 1 a 20. 
Cada um destes valores corresponde a um slot/bloco temporal (de 2 horas).
Representámos apenas os dias da semana, ou seja, teremos 4 blocos por dia.
Com este domínio temos então 20 blocos que correspondem a 20 possíveis atribuições para aulas.

Para além das variáveis disciplinas, temos um conjunto de variáveis auxiliares para tornar o problema um pouco mais realista.
```python
    # Variables related to the classes (Lesi and Leec)
    teachers = {
        'Teacher1': ['D1_T', 'D1_TP', 'D7'],
        'Teacher2': ['D2_T', 'D2_TP', 'D5_T'],
        'Teacher3': ['D5_TP', 'D3', 'D4_T'],
        'Teacher4': ['D4_TP', 'D6_T', 'D6_TP', 'D9_T', 'D10_T'],
        'Teacher5': ['D8_TP', 'D8_T', 'D9_TP', 'D10_TP'],
    }

    # We could have had most of the classes in each class taught in the same classroom
    # But we wanted to test some intersections between Lesi and Leec classes taking place in the same classroom
    classrooms= {
        'room1':['D1_T', 'D5_T', 'D8_T', 'D3'],
        'room2':['D4_T', 'D6_T', 'D7', 'D2_T'],
        'room3':['D9_T', 'D9_TP', 'D10_T', 'D10_TP'],
        'lab1': ['D5_TP', 'D2_TP', 'D4_TP'], 
        'lab2': ['D1_TP', 'D6_TP', 'D8_TP']
    }

    # Some scheduling preferences for teachers (later we can choose to assign 
    # or assign if possible (the closest to the value))
    teacher_preferences = {
        'Teacher1': {'D1_T': 2, 'D1_TP': 6},
        'Teacher3': {'D3': 7, 'D5_TP': 11},
        'Teacher5': {'D8_T': 14}
    }
```


Relativamente às restrições (n-ary), temos que:

- Todas as aulas tem a duração de 2 horas e são leccionadas nos dias da semana.
- Apenas 1 ou 2 aulas por manhã/tarde.

Estas duas restrições são cumpridas pela própria formulação do problema.

- Disciplinas relacionadas a uma turma tomam valores distintos.
```python
    (ScheduleSolver.distinct_val(len(Lesi)), Lesi), # same as AllDifferentConstraint()
    (ScheduleSolver.distinct_val(len(Leec)), Leec), # same as AllDifferentConstraint()
```

- Todas as turmas têm 4 a 10 aulas por semana.

Esta restrição também é cumprida pela própria formulação do problema, mas adicionámos na implementação para verificar o seu cumprimento.
```python
    (lambda *values: ScheduleSolver.atleast_per_week(4, *values), Lesi),
    (lambda *values: ScheduleSolver.atmost_per_week(10, *values), Lesi),
    (lambda *values: ScheduleSolver.atleast_per_week(4, *values), Leec),
    (lambda *values: ScheduleSolver.atmost_per_week(10, *values), Leec)
```

Para as restrições relacionadas com os dias individualmente, utilizámos uma variável, subsets, que é uma lista que contém subsets associados a um dia específico (Ex: Segunda corresponde a {1,2,3,4}).
```python
    subsets = generate_subsets(start=1, end=20, sub_size=4)
```

Para as variáveis que representam a mesma disciplina, mas que diferem apenas no sufixo _T ou_TP, armazenámos numa lista com esses pares em tuplos (Ex: pairs = [('D1_T', 'D1_TP'), ....])
```python
    pairs_TandTp = get_pairs_TandTP(Lesi + Leec)
```

- Uma turma não tem mais de 4 dias de aulas por semana
```python
    (lambda *values, target_values_list=subsets: 
            ScheduleSolver.atmost4_days_per_week(*values, target_values=target_values_list), Lesi),
    (lambda *values, target_values_list=subsets: 
            ScheduleSolver.atmost4_days_per_week(*values, target_values=target_values_list), Leec)
```

- Turma não tem mais de 3 aulas por dia
```python
    for i, subset in enumerate(subsets):
        constraints_to_add.append(
            (lambda *values, target_values=subset: ScheduleSolver.atmost_3lessons_per_day(*values,
            target_values=target_values), Lesi))
        constraints_to_add.append(
            (lambda *values, target_values=subset: ScheduleSolver.atmost_3lessons_per_day(*values,
            target_values=target_values), Leec))
```

- Turma tem mais do que 1 aula por dia (nos dias que tem aulas)
```python
    for i, subset in enumerate(subsets):
        constraints_to_add.append(
            (lambda *values, target_values=subset: ScheduleSolver.atleast_2lessons_per_day(*values,
             target_values=target_values), Lesi))
        constraints_to_add.append(
            (lambda *values, target_values=subset: ScheduleSolver.atleast_2lessons_per_day(*values,
             target_values=target_values), Leec))
```

- Turma tem uma certa disciplina uma vez por dia (_T ou _TP)

Nota: Na implementação este ciclo está inserido no ciclo indicado acima, relativo aos subsets.
    
```python
    for pair in pairs_TandTp:
        constraints_to_add.append(
            (lambda *values, target_values=subset: 
                ScheduleSolver.atmost_1lessonTorTp_perday(*values, target_values=target_values), pair))
```

- Disciplinas, mesmo que de turmas diferentes, que um determinado professor lecciona são atribuídas para slots/blocos diferentes.
```python
    for teacher, subjects in teachers.items():
    constraints_to_add.append((AllDifferentConstraint(), subjects))
```

- Cada professor tem no máximo 3 aulas por dia
```python
    for sub in subsets:
    for teacher, subjects in teachers.items():
        constraints_to_add.append(
            (lambda *values, target_values=sub: ScheduleSolver.atmost_3lessons_per_day(*values,
            target_values=target_values), subjects))
```

- Atribuição de certas preferências de calendarização por parte dos professores.
```python
    # Assign if possible -> minimize the absolute difference, but dont necessarily enforce equality.
    # abs(value - pref_value) <= n
    # Enforce assignment (value == pref_value)
    for teacher, preferences in teacher_preferences.items():
        for subject, preferred_value in preferences.items():
            constraints_to_add.append(
                (lambda value, pref_value=preferred_value: value == pref_value, [subject]))   
```

- Disciplinas associadas a uma mesma sala são atribuídos valores/slots diferentes.
```python
    for classroom, subjects in classrooms.items():
    constraints_to_add.append((AllDifferentConstraint(), subjects))
```

- No final adicionámos uma restrição para tentar minimizar a diferença absoluta entre as atribuições para cada uma das turmas, para tentar diminuir os "gaps" entre aulas. A relevância desta restrição, ou seja, se tem algum impacto prático ou não, será discutida numa secção posterior.
```python
    # Add constraints to try to reduce the gaps
    solver_minconflicts.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Lesi))
    solver_minconflicts.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Leec))
```

As funções implementadas para as restrições supracitadas, podem ser consultadas na classe ScheduleSolver.

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Algoritmos/Heurísticas
</div>

Na resolução do problema utilizámos os algoritmos MinConflicts e BackTracking da biblioteca python-constraints.

MinConflicts é um algoritmo de procura heurístico.
Começa com uma atribuição inicial e depois iterativamente escolhe a variável que está em conflito e atribui a essa variável um valor que minimize o número de conflitos.
Não é completo nem óptimo, pode achar apenas um mínimo local e não um mínimo global.

Backtracking é um algoritmo sistemático que usa a abordagem depth first search no espaço das atribuições possíveis.
Começa por atribuir um valor a uma variável, seguindo para a próxima. Se em qualquer ponto, o algoritmo "percebe" que a atribuição atual não levará a nenhuma solução, volta atrás (backtracks) para o ponto de decisão anterior e efetua uma escolha diferente.
É completo mas não óptimo, podendo ser demorado especialmente quando tem várias escolhas disponíveis.
O algoritmo backtracking da biblioteca python-constraints utiliza uma mistura das heurísticas Degree e Minimum Remaining Values (MRV).

A heurística Degree, considera primeiro variáveis envolvidas no maior número de restrições, de forma a "cortar" o espaço de procura.

A heurística MRV, selecciona a variável com menor número de valores "legais", para reduzir o factor de ramificação e detetar possíveis falhas antecipadamente no processo.

Para além disso, o BackTracking utiliza por defeito, a técnica de inferência forward checking para aumentar a eficiência no processo de procura.
O forward checking reduz o espaço de procura eliminando atribuições inválidas assim que são detetadas.
Após a atribuição de um valor a uma variável, faz a avaliação do domínio restante das outras variáveis para assegurar consistência.


<div style='text-align: center;font-size: 35px;color: #24aaf2;'>
  Implementação
</div>


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Imports
</div>

In [1]:
from constraint import AllDifferentConstraint, BacktrackingSolver, MinConflictsSolver, FunctionConstraint
from ScheduleSolverClass import ScheduleSolver
import random
from customUtils import (get_pairs_TandTP, generate_subsets, save_dict_to_json,
                         add_days_hours_columns, create_classSchedule_dataframe, create_dataframe_from_json)

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Variáveis para adicionar ao problema
</div>

In [2]:
Lesi = ['D1_T', 'D1_TP', 'D2_T', 'D2_TP', 'D3', 'D4_T', 'D4_TP', 'D5_T', 'D5_TP']
Leec = ['D6_T', 'D6_TP', 'D7', 'D8_T', 'D8_TP','D9_T', 'D9_TP', 'D10_T', 'D10_TP']

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Variáveis auxiliares
</div>

In [3]:
# Variables related to the classes (Lesi and Leec)
teachers = {
    'Teacher1': ['D1_T', 'D1_TP', 'D7'],
    'Teacher2': ['D2_T', 'D2_TP', 'D5_T'],
    'Teacher3': ['D5_TP', 'D3', 'D4_T'],
    'Teacher4': ['D4_TP', 'D6_T', 'D6_TP', 'D9_T', 'D10_T'],
    'Teacher5': ['D8_TP', 'D8_T', 'D9_TP', 'D10_TP']
}

# We could have had most of the classes in each class taught in the same classroom
# But we wanted to test some intersections between Lesi and Leec classes taking place in the same classroom
classrooms= {
    'room1':['D1_T', 'D5_T', 'D8_T', 'D3'],
    'room2':['D4_T', 'D6_T', 'D7', 'D2_T'],
    'room3':['D9_T', 'D9_TP', 'D10_T', 'D10_TP'],
    'lab1': ['D5_TP', 'D2_TP', 'D4_TP'], 
    'lab2': ['D1_TP', 'D6_TP', 'D8_TP']
}

# Some scheduling preferences for teachers (later we can choose to assign or assign if possible (the closest to the value))
teacher_preferences = {
    'Teacher1': {'D1_T': 2, 'D1_TP': 6},
    'Teacher3': {'D3': 7, 'D5_TP': 9},
    'Teacher5': {'D8_T': 14}
}


# Variables to help to solve the problem
pairs_TandTp = get_pairs_TandTP(Lesi + Leec)
subsets = generate_subsets(start=1, end=20, sub_size=4)

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Restrições a adicionar ao problema

In [4]:
# List of constraints to add to the solver
constraints_to_add = [
    (ScheduleSolver.distinct_val(len(Lesi)), Lesi), # same as AllDifferentConstraint()
    (ScheduleSolver.distinct_val(len(Leec)), Leec), # same as AllDifferentConstraint()
    (lambda *values: ScheduleSolver.atleast_per_week(4, *values), Lesi),
    (lambda *values: ScheduleSolver.atmost_per_week(10, *values), Lesi),
    (lambda *values: ScheduleSolver.atleast_per_week(4, *values), Leec),
    (lambda *values: ScheduleSolver.atmost_per_week(10, *values), Leec),
    (lambda *values, target_values_list=subsets: 
            ScheduleSolver.atmost4_days_per_week(*values, target_values=target_values_list), Lesi),
    (lambda *values, target_values_list=subsets: 
            ScheduleSolver.atmost4_days_per_week(*values, target_values=target_values_list), Leec)
]

# Atmost 3 lessons per day, atleast 2 lessons per day (if classes in that day) and atmost 1 lesson T or Tp per day
for i, subset in enumerate(subsets):
    constraints_to_add.append(
        (lambda *values, target_values=subset: ScheduleSolver.atmost_3lessons_per_day(*values, target_values=target_values), Lesi))
    constraints_to_add.append(
        (lambda *values, target_values=subset: ScheduleSolver.atmost_3lessons_per_day(*values, target_values=target_values), Leec))
    constraints_to_add.append(
        (lambda *values, target_values=subset: ScheduleSolver.atleast_2lessons_per_day(*values, target_values=target_values), Lesi))
    constraints_to_add.append(
        (lambda *values, target_values=subset: ScheduleSolver.atleast_2lessons_per_day(*values, target_values=target_values), Leec))

    for pair in pairs_TandTp:
        constraints_to_add.append(
            (lambda *values, target_values=subset: 
                ScheduleSolver.atmost_1lessonTorTp_perday(*values, target_values=target_values), pair))

# Lessons lectured by teacher in different time slots
for teacher, subjects in teachers.items():
    constraints_to_add.append((AllDifferentConstraint(), subjects))

# Teachers have atmost 3 lessons per day
for sub in subsets:
    for teacher, subjects in teachers.items():
        constraints_to_add.append(
        (lambda *values, target_values=sub: ScheduleSolver.atmost_3lessons_per_day(*values, target_values=target_values), subjects))
        
# Lessons that take place in the same classroom in different time slots       
for classroom, subjects in classrooms.items():
    constraints_to_add.append((AllDifferentConstraint(), subjects))

# Assign if possible -> minimize the absolute difference, but dont necessarily enforce equality.
# abs(value - pref_value) <= n
# Enforce assignment (value == pref_value)
for teacher, preferences in teacher_preferences.items():
    for subject, preferred_value in preferences.items():
        constraints_to_add.append(
            (lambda value, pref_value=preferred_value: value == pref_value, [subject]))

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Instanciar ScheduleSolver (MinConflictsSolver)
</div>

In [5]:
solver_minconflicts = ScheduleSolver(subjects=Lesi + Leec, teachers=teachers, solver=MinConflictsSolver())
# Add variables
solver_minconflicts.add_variables(Lesi + Leec)
# Add the constraints in the list 
solver_minconflicts.add_constraints(*constraints_to_add)
# Add constraints to try to reduce the gaps
solver_minconflicts.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Lesi))
solver_minconflicts.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Leec))

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Mostrar e guardar resultados
</div>

In [6]:
print("-----------MinConflicts-------------")
solution_minconflicts = solver_minconflicts.solve()

# Show and save results
if solution_minconflicts:
    for var, val in sorted(solution_minconflicts.items(), key=lambda item: item[1]):
        print(f'{val}: {var}')
    # Save to Json file   
    save_dict_to_json(solution_minconflicts, teachers, classrooms, filename="MinConflicts.json")
else:
    print("No solution found!!!")

-----------MinConflicts-------------
2: D1_T
2: D8_TP
3: D9_TP
4: D5_T
4: D6_TP
5: D7
6: D1_TP
7: D3
7: D10_TP
8: D6_T
9: D5_TP
10: D4_T
12: D2_T
13: D9_T
14: D4_TP
14: D8_T
15: D2_TP
16: D10_T


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Instanciar ScheduleSolver (BackTrackingSolver)
</div>

In [7]:
solver_backtracking = ScheduleSolver(subjects=Lesi + Leec, teachers=teachers, solver=BacktrackingSolver(forwardcheck=True))
# Add variables
solver_backtracking.add_variables(Lesi + Leec)
# Add the constraints in the list 
solver_backtracking.add_constraints(*constraints_to_add)
# Add constraints to try to reduce the gaps
solver_backtracking.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Lesi))
solver_backtracking.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Leec))

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Mostrar e guardar resultados
</div>

In [8]:
print("-----------BackTracking(Forward)-------------")
# Get 10 first solutions from iterator
solutionIter_backtracking = solver_backtracking.solve(from_iterator=True, max_solutions=10)
solutions_list = list(solutionIter_backtracking)

# Select one solution randomly from those 10
# Show and save results
if solutions_list:
    random_solution = random.choice(solutions_list)
    for var, val in sorted(random_solution.items(), key=lambda item: item[1]):
        print(f'{val}: {var}')
    # Save to Json file   
    save_dict_to_json(random_solution, teachers, classrooms, filename="BackTracking.json")
else:
    print("No solutions found!")

-----------BackTracking(Forward)-------------
1: D4_TP
2: D1_T
5: D4_T
6: D1_TP
7: D3
9: D5_TP
9: D6_T
11: D7
12: D9_T
12: D2_TP
13: D6_TP
14: D8_T
16: D10_T
18: D9_TP
19: D10_TP
19: D2_T
20: D5_T
20: D8_TP


<div style='text-align: center;font-size: 35px;color: #24aaf2;'>
  Resultados (Dataframes)
</div>

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Variáveis para rearranjar dataframes
</div>

In [9]:
# Map to days and hour-blocks
day_mapping = {
    'Monday': [1, 2, 3, 4],
    'Tuesday': [5, 6, 7, 8],
    'Wednesday': [9, 10, 11, 12],
    'Thursday': [13, 14, 15, 16],
    'Friday': [17, 18, 19, 20]
}
hours_mapping = {
    '9-11h': [1, 5, 9, 13, 17],
    '11-13h': [2, 6, 10, 14, 18],
    '14-16h': [3, 7, 11, 15, 19],
    '16-18h': [4, 8, 12, 16, 20],
}

# Create new dataframe (Schedule-like) for each class
# Define the indexes (rows)
time_indexes = ['9-11h', '11-13h', '14-16h', '16-18h']

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Criar horário semanal para Lesi e Leec (MinConflicts)
</div>

In [10]:
# Dataframe from Json
df_Lesi_mc = create_dataframe_from_json("MinConflicts.json", Lesi)
df_Leec_mc = create_dataframe_from_json("MinConflicts.json", Leec)

# Create a new DataFrame with the content of the original rearranged by time blocks and weekdays
new_df_Lesi_mc = create_classSchedule_dataframe(df_Lesi_mc, time_indexes, day_mapping, hours_mapping)
new_df_Leec_mc = create_classSchedule_dataframe(df_Leec_mc, time_indexes, day_mapping, hours_mapping)

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Criar horário semanal para Lesi e Leec (Backtracking)
</div>

In [11]:
# Dataframe from Json
df_Lesi_back = create_dataframe_from_json("BackTracking.json", Lesi)
df_Leec_back = create_dataframe_from_json("BackTracking.json", Leec)

# Create a new DataFrame with the content of the original rearranged by time blocks and weekdays
new_df_Lesi_back = create_classSchedule_dataframe(df_Lesi_back, time_indexes, day_mapping, hours_mapping)
new_df_Leec_back = create_classSchedule_dataframe(df_Leec_back, time_indexes, day_mapping, hours_mapping)

<div style='text-align: center;font-size: 30px;color: #24aaf2;'>
  Dataframes MinConflicts
</div>

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Horário Semanal
</div>

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Lesi
</div>

In [12]:
new_df_Lesi_mc

,Monday,Tuesday,Wednesday,Thursday,Friday
9-11h,,,D5_TP,,
11-13h,D1_T,D1_TP,D4_T,D4_TP,
14-16h,,D3,,D2_TP,
16-18h,D5_T,,D2_T,,


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Leec
</div>

In [13]:
new_df_Leec_mc

,Monday,Tuesday,Wednesday,Thursday,Friday
9-11h,,D7,,D9_T,
11-13h,D8_TP,,,D8_T,
14-16h,D9_TP,D10_TP,,,
16-18h,D6_TP,D6_T,,D10_T,


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Relação das disciplinas de Lesi com professores e salas
</div>

In [14]:
# Add columns hours and days to the dataframe
add_days_hours_columns(df_Lesi_mc,day_mapping,hours_mapping)

,Slot,Teacher,Classroom,Day,Hour
D1_T,2,Teacher1,room1,Monday,11-13h
D1_TP,6,Teacher1,lab2,Tuesday,11-13h
D2_T,12,Teacher2,room2,Wednesday,16-18h
D2_TP,15,Teacher2,lab1,Thursday,14-16h
D3,7,Teacher3,room1,Tuesday,14-16h
D4_T,10,Teacher3,room2,Wednesday,11-13h
D4_TP,14,Teacher4,lab1,Thursday,11-13h
D5_T,4,Teacher2,room1,Monday,16-18h
D5_TP,9,Teacher3,lab1,Wednesday,9-11h


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Relação das disciplinas de Leec com professores e salas
</div>

In [15]:
# Add columns hours and days to the dataframe
add_days_hours_columns(df_Leec_mc,day_mapping,hours_mapping)

,Slot,Teacher,Classroom,Day,Hour
D6_T,8,Teacher4,room2,Tuesday,16-18h
D6_TP,4,Teacher4,lab2,Monday,16-18h
D7,5,Teacher1,room2,Tuesday,9-11h
D8_T,14,Teacher5,room1,Thursday,11-13h
D8_TP,2,Teacher5,lab2,Monday,11-13h
D9_T,13,Teacher4,room3,Thursday,9-11h
D9_TP,3,Teacher5,room3,Monday,14-16h
D10_T,16,Teacher4,room3,Thursday,16-18h
D10_TP,7,Teacher5,room3,Tuesday,14-16h


<div style='text-align: center;font-size: 30px;color: #24aaf2;'>
  Dataframes BackTracking
</div>

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Horário Semanal
</div>

<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Lesi
</div>

In [16]:
new_df_Lesi_back

,Monday,Tuesday,Wednesday,Thursday,Friday
9-11h,D4_TP,D4_T,D5_TP,,
11-13h,D1_T,D1_TP,,,
14-16h,,D3,,,D2_T
16-18h,,,D2_TP,,D5_T


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Leec
</div>

In [17]:
new_df_Leec_back

,Monday,Tuesday,Wednesday,Thursday,Friday
9-11h,,,D6_T,D6_TP,
11-13h,,,,D8_T,D9_TP
14-16h,,,D7,,D10_TP
16-18h,,,D9_T,D10_T,D8_TP


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Relação das disciplinas de Lesi com professores e salas
</div>

In [18]:
# Add columns hours and days to the dataframe
add_days_hours_columns(df_Lesi_back,day_mapping,hours_mapping)

,Slot,Teacher,Classroom,Day,Hour
D1_T,2,Teacher1,room1,Monday,11-13h
D1_TP,6,Teacher1,lab2,Tuesday,11-13h
D5_TP,9,Teacher3,lab1,Wednesday,9-11h
D5_T,20,Teacher2,room1,Friday,16-18h
D4_TP,1,Teacher4,lab1,Monday,9-11h
D4_T,5,Teacher3,room2,Tuesday,9-11h
D2_T,19,Teacher2,room2,Friday,14-16h
D2_TP,12,Teacher2,lab1,Wednesday,16-18h
D3,7,Teacher3,room1,Tuesday,14-16h


<div style='text-align: left;font-size: 25px;color: #24aaf2;'>
  Relação das disciplinas de Leec com professores e salas
</div>

In [19]:
# Add columns hours and days to the dataframe
add_days_hours_columns(df_Leec_back,day_mapping,hours_mapping)

,Slot,Teacher,Classroom,Day,Hour
D8_T,14,Teacher5,room1,Thursday,11-13h
D8_TP,20,Teacher5,lab2,Friday,16-18h
D10_TP,19,Teacher5,room3,Friday,14-16h
D10_T,16,Teacher4,room3,Thursday,16-18h
D9_TP,18,Teacher5,room3,Friday,11-13h
D9_T,12,Teacher4,room3,Wednesday,16-18h
D6_TP,13,Teacher4,lab2,Thursday,9-11h
D6_T,9,Teacher4,room2,Wednesday,9-11h
D7,11,Teacher1,room2,Wednesday,14-16h


<div style='text-align: center;font-size: 35px;color: #24aaf2;'>
  Conclusão
</div>



Na conclusão deste projeto, podemos referir que conseguimos cumprir de forma relativamente satisfatória os requisitos propostos.

Observámos que ambos os algoritmos, MinConflicts e Backtracking, proporcionaram resultados satisfatórios com ligeiras variações nos horários gerados.

Comparando os dois algoritmos utilizados, verificámos que o MinConflicts é mais rápido e versátil do que o Backtracking.
Na versão apresentada, relativamente às combinações de professores/turmas/preferências de agendamento utilizadas, ambos os algoritmos conseguiram chegar à solução de forma bastante rápida.

Contudo, nos diversos testes efetuados com várias combinações, verificámos que o MinConflicts chegava sempre à solução de forma rápida, enquanto que o BackTracking por vezes demorava bastante tempo a encontrar solução.
Para além disso, se acrescentarmos alguma complexidade no número de variáveis ou restrições, o MinConflicts consegue "lidar" com essas alterações de forma mais eficiente do que o BackTracking.

Quanto à restrição para minimizar a diferença absoluta entre os valores atribuídos, notámos que não tem grande impacto no resultado final, o que pode ser explicado pela forma como a restrição foi implementada.

A restrição referida foi implementada de forma bastante "flexivel" na escolha da incrementação do threshold.
No caso de termos optado por utilizar um limite para o threshold mais "apertado", ambos os algoritmos demoravam bastante tempo a encontrar a solução, sendo que com certas combinações de professores/turmas/preferências de agendamento retornavam que nenhuma solução foi encontrada, ou seja, o problema com aquelas características/restrições não tinha solução para aquele range de threshold.

Caso se pretenda testar diversas combinações, e em casos que os algoritmos retornem que nenhuma solução foi encontrada, pode-se remover essa restrição, eliminando as seguintes linhas de código nos solvers dos dois algoritmos:
```python
    # Add constraints to try to reduce the gaps
    solver_backtracking.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Lesi))
    solver_backtracking.add_constraints((FunctionConstraint(ScheduleSolver.minimize_absolute_difference), Leec))
```